# Train PPO

In [ ]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import gym
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.optim import Adam
from torch.distributions import MultivariateNormal
import time

import random

In [ ]:
def angle_normalize(x):
    return ((x + torch.pi) % (2 * torch.pi)) - torch.pi

def angle_normalize_np(x):
    return ((x + np.pi) % (2 * np.pi)) - np.pi


# The Policy Network

In [ ]:
class Network(nn.Module):
	def __init__(self, size_in, size_out,size_hidden):
		super(Network, self).__init__()
		self.layer1 = nn.Linear(size_in, size_hidden)
		self.layer2 = nn.Linear(size_hidden, size_hidden)
		self.layer3 = nn.Linear(size_hidden, size_out)

	def forward(self, obs):
		# Convert observation to tensor if it's a numpy array
		if isinstance(obs, np.ndarray):
			obs = torch.tensor(obs, dtype=torch.float)
		m = nn.Tanh()
		activation1 = F.relu(self.layer1(obs.float()))
		activation2 = F.relu(self.layer2(activation1))
		output = self.layer3(activation2)
		output = 2 * m(output)
		return output.float()


# Train PPO with Gym

In [ ]:
def PPO(seed = 0, profix = "", timesteps_per_batch = 5000):
    ''' hyperparameter '''
    # collect data
    timesteps_per_batch = timesteps_per_batch   # Number of timesteps to run per batch, episode
    max_timesteps_per_episode = 200             # Max number of timesteps per episode, steps
    total_timesteps = 200                       # collect how many times
    n_updates_per_iteration = 1                 # Number of times to update actor/critic per iteration
    lr = 0.005                                  # Learning rate of actor optimizer
    gamma = 0.95                                # Discount factor to be applied when calculating Rewards-To-Go
    clip = 0.2                                  # Recommended 0.2, helps define the threshold to clip the ratio during SGA
    
    ''' init env '''
    env = gym.make("Pendulum-v1")

    ''' config filenames '''
    profix = profix + "_"
    # filename = f"PPO_{profix}total[{total_timesteps}]_up[{n_updates_per_iteration}]_ep[{timesteps_per_batch}]_step[{max_timesteps_per_episode}]_lr[{lr}]_cp[{clip}]_sd[{seed}]-test"
    filename = f"PPO_GYM_{timesteps_per_batch}_sd{seed}"

    np.random.seed(seed)
    torch.manual_seed(seed) # set random seed
    random.seed(seed)

    ''' init networks '''
    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]        # 1
    
    actor = Network(size_hidden=32, size_in=3,size_out=1)
    critic = Network(size_hidden=32, size_in=3,size_out=1)
    actor_optim = optim.Adam(actor.parameters(), lr=lr)
    critic_optim = optim.Adam(critic.parameters(), lr=lr)
    cov_var = torch.full(size=(act_dim,), fill_value=0.5)
    cov_mat = torch.diag(cov_var)

    all_average_reward = []
    all_variance = []

    ''' methods '''
    def rollout():  # collect data for episode times, each episode contains n steps 
        # Batch data. For more details, check function header.
        batch_obs = []
        batch_acts = []
        batch_log_probs = []
        batch_rews = []
        batch_rtgs = []
        batch_lens = []
        batch_reward_average = []
        reward_all = []

        ep_rews = []

        t = 0 # Keeps track of how many timesteps we've run so far this batch

        # Keep simulating until we've run more than or equal to specified timesteps per batch
        while t < timesteps_per_batch:      # like episode
            ep_rews = [] # rewards collected per episode

            # Reset the environment. sNote that obs is short for observation. 
            obs = env.reset()
            done = False

            # Run an episode for a maximum of max_timesteps_per_episode timesteps
            for ep_t in range(max_timesteps_per_episode):       # like steps
                t += 1 # Increment timesteps ran this batch so far

                # Track observations in this batch
                batch_obs.append(obs)

                # Calculate action and make a step in the env. 
                # Note that rew is short for reward.
                action, log_prob = get_action(obs)
                obs, rew, done, _ = env.step(action)

                # Track recent reward, action, and action log probability
                ep_rews.append(rew)
                reward_all.append(rew)
                batch_acts.append(action)
                batch_log_probs.append(log_prob)

                # If the environment tells us the episode is terminated, break
                if done:
                    break

            # Track episodic lengths and rewards
            batch_lens.append(ep_t + 1)
            batch_rews.append(ep_rews)
            batch_reward_average.append(sum(ep_rews)/len(ep_rews))

        batch_obs = torch.from_numpy(np.array(batch_obs)
        batch_acts = torch.from_numpy(np.array(batch_acts))
        batch_log_probs = torch.from_numpy(np.array(batch_log_probs))

        batch_rtgs = compute_rtgs(batch_rews)
        r_all_average = sum(reward_all) / len(reward_all)
        all_variance.append(torch.tensor(batch_rews).var().item())

        return batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens, r_all_average

    def compute_rtgs(batch_rews):
        # The rewards-to-go (rtg) per episode per batch to return.
        # The shape will be (num timesteps per episode)
        batch_rtgs = []

        # Iterate through each episode
        for ep_rews in reversed(batch_rews):

            discounted_reward = 0 # The discounted reward so far

            # Iterate through all rewards in the episode. We go backwards for smoother calculation of each
            # discounted return (think about why it would be harder starting from the beginning)
            for rew in reversed(ep_rews):
                discounted_reward = rew + discounted_reward * gamma
                batch_rtgs.insert(0, discounted_reward)
        # Convert the rewards-to-go into a tensor
        batch_rtgs = torch.tensor(batch_rtgs, dtype=torch.float)
        return batch_rtgs

    def get_action(obs):
        # Query the actor network for a mean action
        mean = actor(obs)

        # Create a distribution with the mean action and std from the covariance matrix above.
        dist = MultivariateNormal(mean, cov_mat)

        # Sample an action from the distribution
        action = dist.rsample()

        # Calculate the log probability for that action
        log_prob = dist.log_prob(action)

        # Return the sampled action and the log probability of that action in our distribution
        return action.detach().numpy(), log_prob.detach()

    def evaluate(batch_obs, batch_acts):
        V = critic(batch_obs).squeeze()
        mean = actor(batch_obs)
        dist = MultivariateNormal(mean, cov_mat)
        log_probs = dist.log_prob(batch_acts)
        # Return the value vector V of each observation in the batch
        # and log probabilities log_probs of each action in the batch
        return V, log_probs

    print(f"Learning... Running {max_timesteps_per_episode} timesteps per episode, ", end='')
    print(f"{timesteps_per_batch} timesteps per batch for a total of {total_timesteps} timesteps")
    t_so_far = 0 # Timesteps simulated so far
    i_so_far = 0 # Iterations ran so far
    while t_so_far < total_timesteps:                                                                       # ALG STEP 2
        # Autobots, roll out (just kidding, we're collecting our batch simulations here)
        batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens, r_all_average = rollout()                     # ALG STEP 3

        # all_average_reward.append(batch_rtgs.mean().item())
        all_average_reward.append(r_all_average) 

        # Calculate how many timesteps we collected this batch
        t_so_far += 1

        # Increment the number of iterations
        i_so_far += 1

        # Calculate advantage at k-th iteration, V from critic, rtgs from actor
        V, _ = evaluate(batch_obs, batch_acts)
        A_k = batch_rtgs - V.detach()

        A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-10)

        # This is the loop where we update our network for some n epochs
        ''' update n times'''
        for _ in range(n_updates_per_iteration):
            # Calculate V_phi and pi_theta(a_t | s_t)
            V, curr_log_probs = evaluate(batch_obs, batch_acts)

            # Calculate the ratio pi_theta(a_t | s_t) / pi_theta_k(a_t | s_t)
            ratios = torch.exp(curr_log_probs - batch_log_probs)

            # Calculate surrogate losses.
            surr1 = ratios * A_k
            surr2 = torch.clamp(ratios, 1 - clip, 1 + clip) * A_k

            # Calculate actor and critic losses.
            actor_loss = (-torch.min(surr1, surr2)).mean()
            critic_loss = nn.MSELoss()(V, batch_rtgs)

            # Calculate gradients and perform backward propagation for actor network
            actor_optim.zero_grad()
            actor_loss.backward(retain_graph=True)
            actor_optim.step()

            # Calculate gradients and perform backward propagation for critic network
            critic_optim.zero_grad()
            critic_loss.backward()
            critic_optim.step()


        ''' show result '''
        if t_so_far % 100 == 0:
            # print(f"Episode: {t_so_far}/{total_timesteps}, average:{average_score[-1]}")
            print(f"Episode: {t_so_far}/{total_timesteps}, average reward: {all_average_reward[-1]}")

    ''' save result '''
    time_2 = time.time()
    plt.figure(1)
    x = range(1,len(all_average_reward)+1)
    plt.plot(x,all_average_reward,label = 'average rewards')
    plt.axhline(y=sum(all_average_reward)/len(all_average_reward),c='r', ls="--")
    plt.axhline(y=0,c='g', ls="--")
    plt.legend()
    plt.ylim((-9, 1))
    plt.title(filename)
    plt.xlabel('Episodes')
    plt.savefig(filename + '_collect_rewards' + '.jpg')
    plt.show()
    print(filename)
    torch.save(actor, filename+'_actor' + '.pth')  
    torch.save(critic, filename+'_critic' + '.pth')  
    print("---")
    plt.plot(x,all_variance,label = 'all_variance')
    plt.axhline(y=sum(all_variance)/len(all_variance),c='r', ls="--")
    plt.legend()
    plt.title(filename + "_var")
    plt.xlabel('Episodes')
    plt.savefig(filename +'_var.jpg')
    plt.show()

    ''' save data in file '''
    reward_name = filename + "-rew.csv"
    file = open(reward_name, 'w')
    ### s = ";".join([str(x) for x in average_score])
    s = "\n".join([str(x) for x in all_average_reward])
    file.write(s)   # save value function
    file.write("\n")
    file.write("\n")
    file.close()

    var_name = filename + "-var.csv"
    file = open(var_name, 'w')
    # file.write("varience\n")    # save time
    b = "\n".join([str(x) for x in all_variance])
    file.write(b)

    file.close()



In [ ]:
# run training PPO Gym 
for i in range(0,11):
    PPO(seed=i, profix="r-tanh", timesteps_per_batch = 3000)
    PPO(seed=i, profix="r-tanh", timesteps_per_batch = 1000)

# Train PPO with Differentiable Model

In [ ]:
import datetime
from han_pendulum2 import Han_Pendulum2

def PPO_ts(seed = 0, profix = "", timesteps_per_batch = 5000, method = 0):
    ''' hyperparameter '''
    timesteps_per_batch = timesteps_per_batch   # Number of timesteps to run per batch, episode
    max_timesteps_per_episode = 200             # Max number of timesteps per episode, steps
    total_timesteps = 200                       # collect how many times
    n_updates_per_iteration = 1                 # Number of times to update actor/critic per iteration
    lr = 0.005                                  # Learning rate of actor optimizer
    gamma = 0.95                                # Discount factor to be applied when calculating Rewards-To-Go
    clip = 0.2                                  # Recommended 0.2, helps define the threshold to clip the ratio during SGA
    
    ''' init env'''
    env = Han_Pendulum2(seed=seed)

    ''' config filename'''
    profix = profix + "_"
    today = datetime.datetime.now().strftime('%m%d-%H%M')
    filename = f"PPOtanh_{profix}md{method}_total[{total_timesteps}]_up[{n_updates_per_iteration}]_ep[{timesteps_per_batch}]_step[{max_timesteps_per_episode}]_lr[{lr}]_cp[{clip}]_sd[{seed}]"#_{str(today)}"

    np.random.seed(seed)
    torch.manual_seed(seed) # set random seed
    random.seed(seed)

    ''' init networks '''
    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]        # 1
    
    actor = Network(size_hidden=32, size_in=3,size_out=1)
    critic = Network(size_hidden=32, size_in=3,size_out=1)
    actor_optim = optim.Adam(actor.parameters(), lr=lr)
    critic_optim = optim.Adam(critic.parameters(), lr=lr)
    cov_var = torch.full(size=(act_dim,), fill_value=0.5)
    cov_mat = torch.diag(cov_var)

    all_average_reward = []
    all_variance = []
    

    ''' methods '''
    def rollout():  # collect data for episode times, each episode contains n steps 
        batch_obs = []
        batch_acts = []
        batch_log_probs = []
        batch_rews = []
        batch_rtgs = []
        batch_lens = []
        batch_reward_average = []
        reward_all = []

        ep_rews = []

        t = 0 # Keeps track of how many timesteps we've run so far this batch

        while t < timesteps_per_batch:      # like episode
            ep_rews = [] # rewards collected per episode

            obs = env.reset()                                              
            state = torch.tensor([env.state[0], env.state[1]], dtype=torch.float)
            obs = torch.stack([torch.cos(state[0]), torch.sin(state[0]), state[1]])

            for ep_t in range(max_timesteps_per_episode):       # like steps
                t += 1 # Increment timesteps ran this batch so far

                batch_obs.append(obs)
                action, log_prob = get_action(obs)              # log_prob has no gradient
                obs, rew, done, _ = env.step(action)
                # rew = rew.detach()              
                # state = state.detach()          
                # obs = obs.detach()

                ep_rews.append(rew)
                reward_all.append(rew.item())   
                batch_acts.append(action)
                batch_log_probs.append(log_prob)
            batch_lens.append(ep_t + 1)
            batch_rews.append(ep_rews)
            batch_reward_average.append(sum(ep_rews)/len(ep_rews))

        batch_obs = torch.stack(batch_obs)
        batch_acts = torch.stack(batch_acts)#.squeeze()            # previous: numpy
        batch_log_probs = torch.stack(batch_log_probs)
        batch_rtgs = compute_rtgs(batch_rews)
        all_variance.append(torch.tensor(batch_rews).var().item())

        r_all_average = sum(reward_all) / len(reward_all)

        return batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens, r_all_average

    def compute_rtgs(batch_rews):   # not change gradient
        batch_rtgs = []
        for ep_rews in reversed(batch_rews):
            discounted_reward = 0 # The discounted reward so far
            for rew in reversed(ep_rews):
                discounted_reward = rew + discounted_reward * gamma
                batch_rtgs.insert(0, discounted_reward)
        batch_rtgs = torch.stack(batch_rtgs)
        return batch_rtgs

    def get_action(obs):    # log has no G, A: not changed
        ''' obs has gradient? '''
        mean = actor(obs)       
        dist = MultivariateNormal(mean, cov_mat)
        # action = dist.sample()    # TODO: changed
        action = dist.rsample()
        log_prob = dist.log_prob(action)
        return action, log_prob.detach()

    def evaluate(batch_obs, batch_acts):        # not change G
        '''input:
        action has no gradient?
        obs has gradient?
        ''' 
        V = critic(batch_obs).squeeze()
        mean = actor(batch_obs)
        dist = MultivariateNormal(mean, cov_mat)
        log_probs = dist.log_prob(batch_acts)
        return V, log_probs

    print(f"Learning... Running {max_timesteps_per_episode} timesteps per episode, ", end='')
    print(f"{timesteps_per_batch} timesteps per batch for a total of {total_timesteps} timesteps")
    t_so_far = 0 # Timesteps simulated so far
    i_so_far = 0 # Iterations ran so far
    while t_so_far < total_timesteps:                                                                       # ALG STEP 2
        batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens, r_all_average = rollout()                     # ALG STEP 3
        # all_average_reward.append(batch_rtgs.mean().item())
        all_average_reward.append(r_all_average) 
        t_so_far += 1
        i_so_far += 1
        V, _ = evaluate(batch_obs.detach(), batch_acts.detach())    # TODO: obs has no Gradient, action has no Gradient
        A_k = batch_rtgs - V.detach()                                                                       # ALG STEP 5
        A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-10)
        ''' update n times'''
        for _ in range(n_updates_per_iteration):                                                       # ALG STEP 6 & 7
            V, curr_log_probs = evaluate(batch_obs.detach(), batch_acts.detach())
            ratios = torch.exp(curr_log_probs - batch_log_probs)
            
            ''' TODO: Different methods'''
            if method == 0:
                surr1 = ratios * A_k.detach()
                surr2 = torch.clamp(ratios, 1 - clip, 1 + clip) * A_k.detach()
                actor_loss = (-torch.min(surr1, surr2)).mean()

            elif method == 1:
                surr1 = ratios * A_k 
                surr2 = torch.clamp(ratios, 1 - clip, 1 + clip) * A_k
                actor_loss = (-torch.min(surr1, surr2)).mean()

            elif method == 2:
                surr1 = ratios * A_k.detach() + ratios.detach() * A_k 
                surr2 = torch.clamp(ratios, 1 - clip, 1 + clip) * A_k + torch.clamp(ratios, 1 - clip, 1 + clip).detach() * A_k
                actor_loss = (-torch.min(surr1, surr2)).mean()
            elif method == 3:
                surr1 = ratios.detach() * curr_log_probs * A_k.detach() + ratios.detach() * A_k  # 0407 20:57 B
                surr2 = torch.clamp(ratios, 1 - clip, 1 + clip).detach() * curr_log_probs * A_k.detach() + A_k * torch.clamp(ratios, 1 - clip, 1 + clip).detach()
                actor_loss = (-torch.min(surr1, surr2)).mean()
            elif method == 4:
                surr1 = ratios.detach() * (curr_log_probs * A_k.detach() + A_k)  # 0407 20:30 C
                surr2 = torch.clamp(ratios, 1 - clip, 1 + clip).detach() * (curr_log_probs * A_k.detach() + A_k)
                actor_loss = (-torch.min(surr1, surr2)).mean()
            elif method == 5:
                surr1 =  (torch.exp(curr_log_probs) * A_k) / batch_log_probs.detach() # D 040721:23 2310
                actor_loss = (-surr1).mean()           
            elif method == 6:
                actor_loss = (-A_k).mean()           

            critic_loss = nn.MSELoss()(V, batch_rtgs.detach())
            actor_optim.zero_grad()
            # actor_loss.backward(create_graph = True, retain_graph=True) # TODO: 
            actor_loss.backward(retain_graph=True)
            actor_optim.step()

            critic_optim.zero_grad()
            critic_loss.backward()
            critic_optim.step()


        ''' show result '''
        if t_so_far % 100 == 0:
            print(f"Episode: {t_so_far}/{total_timesteps}, average reward: {all_average_reward[-1]}")
    ''' save result '''
    time_2 = time.time()
    plt.figure(1)
    x = range(1,len(all_average_reward)+1)

    plt.plot(x,all_average_reward,label = 'average rewards')
    plt.axhline(y=sum(all_average_reward)/len(all_average_reward),c='r', ls="--")
    plt.axhline(y=0,c='g', ls="--")
    plt.legend()
    plt.ylim((-9, 1))
    plt.title(filename)
    plt.xlabel('Episodes')
    plt.savefig(filename + '_collect_rewards' + '.jpg')
    plt.show()

    plt.plot(x,all_variance,label = 'variance')
    plt.legend()
    plt.title(filename)
    plt.xlabel('Episodes')
    plt.savefig(filename + '_varience' + '.jpg')
    plt.show()
    print(filename)
    torch.save(actor, filename+'_actor' + '.pth')  
    torch.save(critic, filename+'_critic' + '.pth')  
    print("---")

    ''' save data in file '''
    file_name = filename + ".csv"
    file = open(file_name, 'w')
    ## s = ";".join([str(x) for x in average_score])
    s = "\n".join([str(x) for x in all_average_reward])
    file.write(s)   # save value function
    file.write("\n")
    file.write("\n")
    
    s = "\n".join([str(x) for x in all_variance])
    file.write(s)   # save value function
    file.write("\n")
    file.write("\n")

    file.close()



In [ ]:
# train PPO differentiable many times
for i in range(0, 11):
    PPO_ts(seed=i, profix="", timesteps_per_batch = 1000, method=0)
    PPO_ts(seed=i, profix="", timesteps_per_batch = 1000, method=1)
    PPO_ts(seed=i, profix="", timesteps_per_batch = 1000, method=2)
    PPO_ts(seed=i, profix="", timesteps_per_batch = 1000, method=3)
    PPO_ts(seed=i, profix="", timesteps_per_batch = 1000, method=4)
    # PPO_ts(seed=i, profix="", timesteps_per_batch = 5000, method=5)
    # PPO_ts(seed=i, profix="", timesteps_per_batch = 5000, method=6)